# 스팸 메일 분류

In [60]:
import pandas as pd
data = pd.read_csv('spam.csv', encoding='latin1')

In [ ]:
# 총 데이터 수 확인
print(len(data))

# 데이터 5행 출력
data.head()

In [ ]:
# 필요없는 컬럼 삭제 및 레이블 매핑
data = data.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
data['v1'] = data['v1'].map({'ham': 0, 'spam': 1})

data.head()

In [ ]:
# 결측치 확인
print(data.isnull().sum())

In [ ]:
# 중복된 데이터가 있는지 확인(고유한 데이터와 전체 데이터 개수 비교)
len(data['v2'].unique())

In [67]:
# v2 열에서 중복인 내용이 있다면 중복 제거
data.drop_duplicates(subset=['v2'], inplace=True)

# 총 데이터 수 출력
print('제거 후 남은 샘플 수: ', len(data))

In [ ]:
# 데이터 분포 확인
data['v1'].value_counts()

In [ ]:
# 정상 메일과 스팸 메일의 비율 비교
print(f'정상 메일의 비율 = {round(data["v1"].value_counts()[0]/len(data) * 100,3)}%')
print(f'스팸 메일의 비율 = {round(data["v1"].value_counts()[1]/len(data) * 100,3)}%')

레이블이 굉장히 불균형하기 때문에 훈련 데이터와 테스트 데이터의 레이블 비율을 유지해서 분리

In [ ]:
# X_data, y_data 분리
X_data = data['v2']
y_data = data['v1']

In [88]:
# X_data와 y_data를 8:2 비율로 분할(stratify=y_data)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, stratify=y_data)

In [ ]:
print('훈련 데이터의 비율')
print(f'정상 메일의 비율 = {round(y_train.value_counts()[0]/len(y_train) * 100,3)}%')
print(f'스팸 메일의 비율 = {round(y_train.value_counts()[1]/len(y_train) * 100,3)}%')

In [ ]:
print('테스트 데이터의 비율')
print(f'정상 메일의 비율 = {round(y_test.value_counts()[0]/len(y_test) * 100,3)}%')
print(f'스팸 메일의 비율 = {round(y_test.value_counts()[1]/len(y_test) * 100,3)}%')

In [76]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenizer 객체 생성
tokenizer = Tokenizer()

# X_train에 존재하는 단어마다 부여될 숫자 계산
tokenizer.fit_on_texts(X_train)

# 단어 벡터화
X_train_encoded = tokenizer.texts_to_sequences(X_train)
X_test_encoded = tokenizer.texts_to_sequences(X_test)

In [ ]:
# X_train_encoded 출력
X_train_encoded

In [ ]:
# mapping 된 단어 확인
word_to_index = tokenizer.word_index
print(word_to_index)

In [ ]:
# 단어 집합의 크기를, 전체 단어수+1로 지정
vocab_size = len(word_to_index) + 1
print('단어 집합의 크기: {}'.format((vocab_size)))

In [ ]:
import matplotlib.pyplot as plt

print('메일의 최대 길이 : %d' % max(len(l) for l in X_train_encoded))
print('메일의 평균 길이 : %f' % (sum(map(len, X_train_encoded))/len(X_train_encoded)))
plt.hist([len(s) for s in X_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
# 메일의 최대 길이가 그리 크지 않기 때문에, 최대길이 사용하여 padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 189
X_train_padded = pad_sequences(X_train_encoded, maxlen=max_len)
X_test_padded = pad_sequences(X_test_encoded, maxlen=max_len)
X_train_padded

# 스팸 메일 분류

In [83]:
# 모델 생성 및 컴파일
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Conv1D

model = Sequential()
model.add(Embedding(vocab_size, 32))
model.add(Conv1D(32, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# 모델 학습(history 저장)
history = model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_split=0.2)

In [ ]:
# 테스트 정확도 출력
print('테스트 정확도:',model.evaluate(X_test_padded, y_test)[1])

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()